In [1]:
import itertools
import pandas as pd
from collections import defaultdict 
import pybedtools
from commandtemplate.conda import run_template_bash
from biodata.bigwig import BigWigIReader
from biodata.bed import BEDXReader, BED3Reader, ENCODENarrowPeakReader
from biodata.delimited import DelimitedReader, DelimitedWriter
from genomictools import GenomicCollection, GenomicPos

In [2]:
PROJECT_DIR_d = "/fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/PROcap/"
PROJECT_DIR_r = "/fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/resources/"
PROJECT_DIR_o = "/fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/output/"
# conda_env = "DI"
conda_env = "base"

# Reformat peak call files


In [3]:
def filter_and_reformat_PINTS_divergent_peaks(i, o, bwpl, bwmn, chroms):
	with BigWigIReader(bwpl) as pl, BigWigIReader(bwmn) as mn, BEDXReader(i, ["conf", "fwdTSS", "revTSS"], x=3) as br, DelimitedWriter(o) as dw:
		for bed in br:
			if bed.chrom not in chroms:
				continue
			# check which strand has more read counts
			strand = "+" if pl.value(bed, method="abssum") >= mn.value(bed, method="abssum") else "-"
			dw.write([bed.chrom, bed.chromStart, bed.chromEnd, bed.revTSS, bed.fwdTSS, strand])

In [4]:
def filter_and_reformat_PINTS_unidirectional_peaks(i, o, chroms):
	with BEDXReader(i, ["name", "q", "strand", "read_count", "summit_position", "summit_height"], x=3) as br, DelimitedWriter(o) as dw:
		for bed in br:
			if bed.chrom not in chroms:
				continue
			dw.write([bed.chrom, bed.chromStart, bed.chromEnd, bed.summit_position, ".", bed.strand])

In [5]:
fnames = {"C1": "brm_C1a_and_C1b_erm", 
		  "HCT116": "brm_CTCF_U1_and_CTCF_U2_erm",
		  "HEK293T": "brm_ZNF143_U1_and_ZNF143_U2_erm"
		 }
samples = list(fnames)

incl_chroms = {
	# K562: a female with chronic myelogenous leukemia
	"C1": [f"chr{n}" for n in range(1,23)] + ["chrX"],
	# HCT116: a male with colorectal carcinoma 
	"HCT116": [f"chr{n}" for n in range(1,23)] + ["chrX", "chrY"],
	# HEK293T: human embryonic kidney cells from a female fetus
	"HEK293T": [f"chr{n}" for n in range(1,23)] + ["chrX"]
}

In [6]:
for sample in samples:
	peaktype = "divergent"
	filter_and_reformat_PINTS_divergent_peaks(
		f"{PROJECT_DIR_d}Peaks/PINTS/{fnames[sample]}_1_{peaktype}_peaks.bed",
		f"{PROJECT_DIR_d}peak_reformat/{sample}_{peaktype}.bed",
		f"{PROJECT_DIR_d}Alignments/{fnames[sample]}_5pl.bw",
		f"{PROJECT_DIR_d}Alignments/{fnames[sample]}_5mn.bw",
		incl_chroms[sample]
		)
	peaktype = "unidirectional"
	filter_and_reformat_PINTS_unidirectional_peaks(
		f"{PROJECT_DIR_d}Peaks/PINTS/{fnames[sample]}_1_{peaktype}_peaks.bed",
		f"{PROJECT_DIR_d}peak_reformat/{sample}_{peaktype}.bed",
		incl_chroms[sample]
	)

In [19]:
# Downsampled dataset

samples2 = [f"C1_{n}M" for n in [30, 20, 10, 5, 1]]
for sample in samples2:
	peaktype = "divergent"
	filter_and_reformat_PINTS_divergent_peaks(
		f"{PROJECT_DIR_d}Analysis/{sample}_1_{peaktype}_peaks.bed",
		f"{PROJECT_DIR_d}peak_reformat/{sample}_{peaktype}.bed",
		f"{PROJECT_DIR_d}Analysis/{sample}_5pl.bw",
		f"{PROJECT_DIR_d}Analysis/{sample}_5mn.bw",
		incl_chroms["C1"]
		)
	peaktype = "unidirectional"
	filter_and_reformat_PINTS_unidirectional_peaks(
		f"{PROJECT_DIR_d}Analysis/{sample}_1_{peaktype}_peaks.bed",
		f"{PROJECT_DIR_d}peak_reformat/{sample}_{peaktype}.bed",
		incl_chroms["C1"]
	)

# Peak categories

In [7]:
peaktypes = ["divergent", "unidirectional"]

In [8]:
for sample, peaktype in itertools.product(samples, peaktypes):
	# Proximal
	run_template_bash(
		"biodatatools filter_bed -i {i} -o {o} -bedx {bedx} -overlap_regions {r}",
		conda_env=conda_env,
		i=f"{PROJECT_DIR_d}peak_reformat/{sample}_{peaktype}.bed",
		r=f"{PROJECT_DIR_r}genomes/human/gencode.v37.annotation.TSS.500.bed.bgz",
		o=f"{PROJECT_DIR_d}bed_pd/{sample}_{peaktype}_proximal.bed", 
		bedx=3
		)
	# Distal
	run_template_bash(
		"biodatatools filter_bed -i {i} -o {o} -bedx {bedx} -non_overlap_regions {r}",
		conda_env=conda_env,
		i=f"{PROJECT_DIR_d}peak_reformat/{sample}_{peaktype}.bed",
		r=f"{PROJECT_DIR_r}genomes/human/gencode.v37.annotation.TSS.500.bed.bgz",
		o=f"{PROJECT_DIR_d}bed_pd/{sample}_{peaktype}_distal.bed", 
		bedx=3
		)
	# Distal Intragenic
	run_template_bash(
		"biodatatools filter_bed -i {i} -o {o} -bedx {bedx} -overlap_regions {r}",
		conda_env=conda_env,
		i=f"{PROJECT_DIR_d}bed_pd/{sample}_{peaktype}_distal.bed", 
		r=f"{PROJECT_DIR_r}genomes/human/gencode.v37.annotation.transcripts.union.bed.bgz",
		o=f"{PROJECT_DIR_d}bed_pd/{sample}_{peaktype}_distal_intragenic.bed",
		bedx=3
		)
	# Distal Intergenic
	run_template_bash(
		"biodatatools filter_bed -i {i} -o {o} -bedx {bedx} -non_overlap_regions {r}",
		conda_env=conda_env,
		i=f"{PROJECT_DIR_d}bed_pd/{sample}_{peaktype}_distal.bed", 
		r=f"{PROJECT_DIR_r}genomes/human/gencode.v37.annotation.transcripts.union.bed.bgz",
		o=f"{PROJECT_DIR_d}bed_pd/{sample}_{peaktype}_distal_intergenic.bed", 
		bedx=3
		)

In [22]:
# Downsampled dataset

for sample, peaktype in itertools.product(samples2, peaktypes):
	# Proximal
	run_template_bash(
		"biodatatools filter_bed -i {i} -o {o} -bedx {bedx} -overlap_regions {r}",
		conda_env=conda_env,
		i=f"{PROJECT_DIR_d}peak_reformat/{sample}_{peaktype}.bed",
		r=f"{PROJECT_DIR_r}genomes/human/gencode.v37.annotation.TSS.500.bed.bgz",
		o=f"{PROJECT_DIR_d}bed_pd/{sample}_{peaktype}_proximal.bed", 
		bedx=3
		)
	# Distal
	run_template_bash(
		"biodatatools filter_bed -i {i} -o {o} -bedx {bedx} -non_overlap_regions {r}",
		conda_env=conda_env,
		i=f"{PROJECT_DIR_d}peak_reformat/{sample}_{peaktype}.bed",
		r=f"{PROJECT_DIR_r}genomes/human/gencode.v37.annotation.TSS.500.bed.bgz",
		o=f"{PROJECT_DIR_d}bed_pd/{sample}_{peaktype}_distal.bed", 
		bedx=3
		)

conda run -n DI_test /bin/bash -c "biodatatools filter_bed -i /fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/PROcap/peak_reformat/C1_30M_divergent.bed -o /fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/PROcap/bed_pd/C1_30M_divergent_proximal.bed -bedx 3 -overlap_regions /fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/resources/genomes/human/gencode.v37.annotation.TSS.500.bed.bgz"
conda run -n DI_test /bin/bash -c "biodatatools filter_bed -i /fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/PROcap/peak_reformat/C1_30M_divergent.bed -o /fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/PROcap/bed_pd/C1_30M_divergent_distal.bed -bedx 3 -non_overlap_regions /fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/resources/genomes/human/gencode.v37.annotation.TSS.500.bed.bgz"
conda run -n DI_test /bin/bash -c "biodatatools filter_bed -i /fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/PROcap/peak_ref

# Resize and filter peaks

In [9]:
chrom_sizes = f"{PROJECT_DIR_r}genomes/human/hg38.chrom.sizes.filtered"
blacklist = f"{PROJECT_DIR_r}genomes/human/hg38-blacklist.v2.bed.gz"
ACC_files = {"C1": f"{PROJECT_DIR_r}ENCODE/ENCFF185XRG.bed.gz",
			 "HCT116": f"{PROJECT_DIR_r}ENCODE/ENCFF240LRP.bed.gz",
			 "HEK293T": f"{PROJECT_DIR_r}other/GSE266490/Combined_ATAC_peaks.bed.gz"
			}
distypes = ["distal", "proximal"]

In [21]:
# PINTS: chromStart 0-based, chromEnd 0-based, bed fwdTSS 0-based; bed revTSS 0-based
# biodata.bed BED: chromStart 0-based chromEnd 1-based; BED.genomic_pos --> GenomicPos
# GenomicPos: start 1-based, stop 1-based; ostart 1-based, ostop 1-based; zstart 0-based, zstop 0-based

def resize_and_filter_reformatted_peaks(iuni, idiv, ouni, odiv, ori_uni, ori_di, acc, g, excluded, extension, sigValue=True):	
	if sigValue:
		acc_regions = ENCODENarrowPeakReader.read_all(GenomicCollection, acc)
	else:
		acc_regions = BED3Reader.read_all(GenomicCollection, acc)
	chrom_size = DelimitedReader.read_all(lambda ds: {d[0]:int(d[1]) for d in ds}, g)
	di_regions = BED3Reader.read_all(GenomicCollection, ori_di)
	uni_regions = BED3Reader.read_all(GenomicCollection, ori_uni)
	exc_regions = BED3Reader.read_all(GenomicCollection, excluded)

	kept_regions = defaultdict(dict)
	stats = {}
	for i, k in [[iuni, "uni"], [idiv, "div"]]:
		stats[k] = defaultdict(int)
		# For convinence, we used the same naming scheme here 
		# For unidirectional elements, "revTSS" actually refers to "summit_position" seen above, while "fwdTSS" is set as "."
		with BEDXReader(i, ["revTSS", "fwdTSS", "strand"], x=3) as br:
			for bed in br:
				# Filter 1: find anchor point for resizing
				stats[k]["0. Original"] += 1
				# Unidirectional: anchor on the center of overlapping open chromatin peaks
				if bed.fwdTSS == ".": 
					hits = list(acc_regions.find_overlaps(bed))
					# U1: overlap with DNase peaks
					if len(hits) == 0:
						continue
					stats[k]["0.1. Unidirectional - Found ACC Hits"] += 1
					# U2: the center of ACC peak must be appropriately positioned  (on the left of the prominent TSS if the unidirectional element is on the forward strand, or on the right if on the reverse strand)
					filtered_hits = []
					for hit in hits:
						center = (hit.genomic_pos.ostart + hit.genomic_pos.ostop) // 2
						if not((bed.strand == "+" and (center < int(bed.revTSS))) or (bed.strand == "-" and (center > int(bed.revTSS)))):
							continue
						filtered_hits.append(hit)
					if len(filtered_hits) == 0:
						continue
					stats[k]["0.2. Unidirectional - Good ACC Hits"] += 1
					# U3: if overlapping with multiple hits, retain the one with the highest signal
					if len(filtered_hits) == 1:
						stats[k]["0.3. Unidirectional - Single Good ACC Hits"] += 1
					if sigValue:
						hit = max(filtered_hits, key=lambda h: (min(h.genomic_pos.stop, bed.genomic_pos.stop) - max(h.genomic_pos.start, bed.genomic_pos.start) + 1, h.signalValue))
					else:
						hit = max(filtered_hits, key=lambda h: (min(h.genomic_pos.stop, bed.genomic_pos.stop) - max(h.genomic_pos.start, bed.genomic_pos.start) + 1))
					center = (hit.genomic_pos.ostart + hit.genomic_pos.ostop) // 2
				# Divergent: anchor on the midpoint between two prominent TSSs
				else:
					stats[k]["0.1. Unidirectional - Found ACC Hits"] = "-"
					stats[k]["0.2. Unidirectional - Good ACC Hits"] = "-"
					stats[k]["0.3. Unidirectional - Single Good ACC Hits"] = "-"
					if int(bed.revTSS) >= int(bed.fwdTSS):
						continue
					center = (int(bed.fwdTSS)+int(bed.revTSS)) // 2 

				region = GenomicPos(bed.chrom, center-extension, center+extension)
				stats[k]["1. Found centers"] += 1
				
				# Filter 2: elements not extending beyond chromosome ends
				if region.stop > chrom_size[region.name] or region.start < 1:
					continue
				stats[k]["2. Not extending beyond chromosome ends"] += 1
				
				# Filter 3: not overlapping with ENCODE blacklist regions
				if exc_regions.overlaps(region):
					continue
				stats[k]["3. Not overlapping with ENCODE blacklist regions"] += 1
				
				# Filter 4: original peak region fully contained within the resized region
				if not bed.genomic_pos in region:
					continue
				stats[k]["4. Original peak region fully contained within the resized region"] += 1
				
				# Filter 5: not overlapping with any other elements (the original boundary of all divergent and unidirectional peaks called by PINTS)
				if len(list(uni_regions.find_overlaps(region)) + list(di_regions.find_overlaps(region))) > 1:
					continue
				stats[k]["5. Not overlapping with any other elements (original)"] += 1

				kept_regions[k][region] = bed

	# All resized elements
	all_regions = GenomicCollection([region for k in kept_regions for region in kept_regions[k]])

	for o, k in [[ouni, "uni"], [odiv, "div"]]:
		# Filter 6: not overlapping wtih any other resized elements
		kept = [region for region in kept_regions[k] if len(list(all_regions.find_overlaps(region))) == 1]
		stats[k]["6. Not overlapping with any other elements (resized)"] = len(kept)
		
		with DelimitedWriter(o) as dw:
			for region, bed in kept_regions[k].items():
				if region in kept:
					dw.write([region.name, region.zstart, region.ostop, bed.chromStart, bed.chromEnd, bed.revTSS, bed.fwdTSS, bed.strand])
		
		# Stats for each filtering step
		with DelimitedWriter(o + ".stat.tsv") as dw:
			for i in stats[k].items():
				dw.write(i)

In [22]:
for sample, distype in itertools.product(samples, distypes):
	sigValue = True if sample != "HEK293T" else False
	resize_and_filter_reformatted_peaks(
		f"{PROJECT_DIR_d}bed_pd/{sample}_unidirectional_{distype}.bed", 
		f"{PROJECT_DIR_d}bed_pd/{sample}_divergent_{distype}.bed", 
		f"{PROJECT_DIR_d}bed_plot/{sample}_unidirectional_{distype}.bed", 
		f"{PROJECT_DIR_d}bed_plot/{sample}_divergent_{distype}.bed", 
		f"{PROJECT_DIR_d}Peaks/PINTS/{fnames[sample]}_1_unidirectional_peaks.bed",
		f"{PROJECT_DIR_d}Peaks/PINTS/{fnames[sample]}_1_divergent_peaks.bed",
		ACC_files[sample],
		chrom_sizes,
		blacklist,
		250,
		sigValue
	)

In [23]:
dfs = defaultdict(dict)
for sample, peaktype, distype in itertools.product(samples, peaktypes, distypes):
	dfs[sample][f"{peaktype}_{distype}"] = pd.read_csv(f"{PROJECT_DIR_d}bed_plot/{sample}_{peaktype}_{distype}.bed.stat.tsv", sep="\t", index_col=0, header=None, names=[f"{peaktype}_{distype}"])

In [24]:
pd.concat(list(dfs["C1"].values()), axis=1).sort_index()

,divergent_distal,divergent_proximal,unidirectional_distal,unidirectional_proximal
0. Original,11966,22065,12946,6400
0.1. Unidirectional - Found ACC Hits,-,-,6606,3298
0.2. Unidirectional - Good ACC Hits,-,-,4750,2279
0.3. Unidirectional - Single Good ACC Hits,-,-,4736,2271
1. Found centers,11934,21916,4750,2279
2. Not extending beyond chromosome ends,11934,21916,4750,2279
3. Not overlapping with ENCODE blacklist regions,11854,21729,4710,2260
4. Original peak region fully contained within the resized region,11358,17672,4677,2200
5. Not overlapping with any other elements (original),8960,10216,3669,1443
6. Not overlapping with any other elements (resized),8416,9026,3557,1304


In [25]:
pd.concat(list(dfs["HCT116"].values()), axis=1).sort_index()

,divergent_distal,divergent_proximal,unidirectional_distal,unidirectional_proximal
0. Original,12159,22661,6749,3614
0.1. Unidirectional - Found ACC Hits,-,-,2261,833
0.2. Unidirectional - Good ACC Hits,-,-,1835,656
0.3. Unidirectional - Single Good ACC Hits,-,-,1833,653
1. Found centers,12109,22344,1835,656
2. Not extending beyond chromosome ends,12109,22344,1835,656
3. Not overlapping with ENCODE blacklist regions,12044,22167,1828,651
4. Original peak region fully contained within the resized region,11304,16321,1812,598
5. Not overlapping with any other elements (original),8811,9068,1474,389
6. Not overlapping with any other elements (resized),8378,8264,1445,372


In [26]:
pd.concat(list(dfs["HEK293T"].values()), axis=1).sort_index()

,divergent_distal,divergent_proximal,unidirectional_distal,unidirectional_proximal
0. Original,8735,24505,10037,6392
0.1. Unidirectional - Found ACC Hits,-,-,5951,3627
0.2. Unidirectional - Good ACC Hits,-,-,3832,2027
0.3. Unidirectional - Single Good ACC Hits,-,-,3820,2018
1. Found centers,8694,24261,3832,2027
2. Not extending beyond chromosome ends,8694,24261,3832,2027
3. Not overlapping with ENCODE blacklist regions,8616,24092,3832,2025
4. Original peak region fully contained within the resized region,8146,18839,3818,1957
5. Not overlapping with any other elements (original),5729,10082,2970,1144
6. Not overlapping with any other elements (resized),5392,8949,2887,1028


In [16]:
# For SuppFig1b, distal elements are further divided into intergenic and intragenic

sample = "C1"
for peaktype in peaktypes:
	# Distal Intragenic
	run_template_bash(
		"biodatatools filter_bed -i {i} -o {o} -bedx {bedx} -overlap_regions {r}",
		conda_env=conda_env,
		i=f"{PROJECT_DIR_d}bed_plot/{sample}_{peaktype}_distal.bed", 
		r=f"{PROJECT_DIR_r}genomes/human/gencode.v37.annotation.transcripts.union.bed.bgz",
		o=f"{PROJECT_DIR_d}bed_plot/{sample}_{peaktype}_distal_intragenic.bed",
		bedx=3
	)
	# Distal Intergenic
	run_template_bash(
		"biodatatools filter_bed -i {i} -o {o} -bedx {bedx} -non_overlap_regions {r}",
		conda_env=conda_env,
		i=f"{PROJECT_DIR_d}bed_plot/{sample}_{peaktype}_distal.bed", 
		r=f"{PROJECT_DIR_r}genomes/human/gencode.v37.annotation.transcripts.union.bed.bgz",
		o=f"{PROJECT_DIR_d}bed_plot/{sample}_{peaktype}_distal_intergenic.bed", 
		bedx=3
	)

In [23]:
# Downsampled dataset

for sample, distype in itertools.product(samples2, distypes):
	resize_and_filter_reformatted_peaks(
		f"{PROJECT_DIR_d}bed_pd/{sample}_unidirectional_{distype}.bed", 
		f"{PROJECT_DIR_d}bed_pd/{sample}_divergent_{distype}.bed", 
		f"{PROJECT_DIR_d}bed_plot/{sample}_unidirectional_{distype}.bed", 
		f"{PROJECT_DIR_d}bed_plot/{sample}_divergent_{distype}.bed", 
		f"{PROJECT_DIR_d}Analysis/{sample}_1_unidirectional_peaks.bed",
		f"{PROJECT_DIR_d}Analysis/{sample}_1_divergent_peaks.bed",
		ACC_files["C1"],
		chrom_sizes,
		blacklist,
		250
	)

# Generate control

In [14]:
excl_regions = f"{PROJECT_DIR_r}genomes/human/excluded_regions.bed.gz"
for seed_root, (sample, peaktype, distype) in enumerate(itertools.product(samples, peaktypes, distypes)):
	run_template_bash(
		"cut -f 1-3 {i} | bedtools shuffle -i stdin -g {g} -excl {excl} -seed {seed} -chrom -noOverlapping | bedtools sort -i stdin > {o}",
		conda_env=conda_env,
		i=f"{PROJECT_DIR_d}bed_plot/{sample}_{peaktype}_{distype}.bed",
		g=chrom_sizes,
		excl=excl_regions,
		seed=seed_root+102390,
		o=f"{PROJECT_DIR_d}bed_plot/{sample}_{peaktype}_{distype}_control.bed",
	)